In [ ]:
import copy
import cv2
import os
import rootutils
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d

rootutils.setup_root(
    os.path.abspath(''), indicator=['.git', 'pyproject.toml'], pythonpath=True
)

from src.utils import remove_small_clusters

#### Load pointclouds

In [ ]:
target_pcd = o3d.io.read_point_cloud('../output/pcd/20250429_105703_0.ply')
target_left_img = cv2.imread('../data/left/left_20250429_105703_0.png')

source_pcd = o3d.io.read_point_cloud('../output/pcd/20250429_110523_21.ply')
source_left_img = cv2.imread('../data/left/left_20250429_110523_21.png')

#### Filter pointclouds

In [ ]:
target_pcd = remove_small_clusters(target_pcd, voxel_size=0.01, min_cluster_size=200)
o3d.visualization.draw_geometries([target_pcd])
source_pcd = remove_small_clusters(source_pcd, voxel_size=0.01, min_cluster_size=200)
o3d.visualization.draw_geometries([source_pcd])

#### Align pointclouds

In [ ]:
def preprocess_pointcloud(pcd, voxel_size=0.05):
    """Preprocess the pointcloud: downsample and estimate normals."""
    pcd_down = pcd.voxel_down_sample(voxel_size)
    
    if not pcd_down.has_normals():
        pcd_down.estimate_normals()
        pcd_down.orient_normals_consistent_tangent_plane(k=30)
    
    return pcd_down

def compute_fpfh_features(pcd, voxel_size):
    """Compute FPFH features for registration."""
    return o3d.pipelines.registration.compute_fpfh_feature(
        pcd,
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=100)
    )

def execute_global_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    """Perform global registration using RANSAC."""
    distance_threshold = voxel_size * 1.5
    
    # Define RANSAC parameters
    ransac_n = 3  # Number of points to use for estimation
    checkers = [
        o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
        o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
    ]
    criteria = o3d.pipelines.registration.RANSACConvergenceCriteria(
        max_iteration=100000,
        confidence=0.999
    )
    
    # Execute global registration
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source, target, source_fpfh, target_fpfh, 
        mutual_filter=False,
        max_correspondence_distance=distance_threshold,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        ransac_n=ransac_n, 
        checkers=checkers,
        criteria=criteria
    )
    
    print(f"Global registration result: {result}")
    print(f"Fitness: {result.fitness}, Inlier RMSE: {result.inlier_rmse}")
    return result

def refine_registration(source, target, result_ransac, voxel_size):
    """Refine registration using ICP."""
    distance_threshold = voxel_size * 0.4
    
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, result_ransac.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=10000)
    )
    
    print(f"ICP refinement result: {result}")
    print(f"Fitness: {result.fitness}, Inlier RMSE: {result.inlier_rmse}")
    return result

def align_pointclouds(source, target, voxel_size=0.05):
    """Main function to align two pointclouds."""
    
    # Visualize original pointclouds
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0, 0])  # Red for source
    target_temp.paint_uniform_color([0, 1, 0])  # Green for target
    o3d.visualization.draw_geometries([source_temp, target_temp], 
                                     window_name="Original Pointclouds")
    
    # Preprocess pointclouds
    source_down = preprocess_pointcloud(source, voxel_size)
    target_down = preprocess_pointcloud(target, voxel_size)
    
    # Compute FPFH features
    source_fpfh = compute_fpfh_features(source_down, voxel_size)
    target_fpfh = compute_fpfh_features(target_down, voxel_size)
    
    # Global registration
    result_ransac = execute_global_registration(
        source_down, target_down, source_fpfh, target_fpfh, voxel_size
    )
    
    # Visualize after global registration
    source_temp = copy.deepcopy(source)
    source_temp.transform(result_ransac.transformation)
    source_temp.paint_uniform_color([1, 0, 0])  # Red for source
    target_temp = copy.deepcopy(target)
    target_temp.paint_uniform_color([0, 1, 0])  # Green for target
    print("Displaying alignment after global registration...")
    o3d.visualization.draw_geometries([source_temp, target_temp], 
                                     window_name="After Global Registration")
    
    # Refine registration with ICP
    result_icp = refine_registration(
        source_down, target_down, result_ransac, voxel_size
    )
    
    # Transform the original source pointcloud
    source.transform(result_icp.transformation)
    
    # Visualize final result
    source_temp = copy.deepcopy(source)
    source_temp.paint_uniform_color([1, 0, 0])  # Red for source
    target_temp = copy.deepcopy(target)
    target_temp.paint_uniform_color([0, 1, 0])  # Green for target
    print("Displaying final alignment...")
    o3d.visualization.draw_geometries([source_temp, target_temp], 
                                     window_name="Final Alignment")
    
    return source, result_icp.transformation

In [ ]:
# Voxel size for downsampling
voxel_size = 0.05

# Align pointclouds
aligned_source, transformation = align_pointclouds(source_pcd, target_pcd, voxel_size)
print("Alignment complete. Transformation matrix:")
print(transformation)

#### Compute pcd differences

In [ ]:
def compute_pcd_differences(source: o3d.geometry.PointCloud,
                       reference: o3d.geometry.PointCloud,
                       max_dist: float = None) -> o3d.geometry.PointCloud:
    """
    Colors the `source` cloud based on distances to `reference`.
    Useful for heatmap-like visualization.
    """
    distances = np.asarray(source.compute_point_cloud_distance(reference))
    if max_dist is None:
        max_dist = np.percentile(distances, 95)  # Robust cap to remove outliers

    # Normalize and convert to colormap
    normalized = np.clip(distances / max_dist, 0, 1)
    cmap = plt.get_cmap("jet")
    colors = cmap(normalized)[:, :3]  # Remove alpha

    source_colored = copy.deepcopy(source)
    source_colored.colors = o3d.utility.Vector3dVector(colors)
    return source_colored

In [ ]:
distance_pcd = compute_pcd_differences(aligned_source, target_pcd, max_dist=0.01)
o3d.visualization.draw_geometries([distance_pcd], window_name="Anomaly Map")